# Проект по SQL

## Введение

Компания купила крупный сервис для чтения книг по подписке. <br>
В моем распоряжении 5 баз данных - информация о книгах, авторах, издательствах, оценках и обзорах.

Цель исследования: Проанализировать базу данных, посмотреть взаимосвязи и помочь компании сформулировать ценностное предложение для нового продукта

## Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
query  = '''SELECT * FROM books'''
pd.io.sql.read_sql(query, con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query  = '''SELECT * FROM authors'''
pd.io.sql.read_sql(query, con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query  = '''SELECT * FROM ratings'''
pd.io.sql.read_sql(query, con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query  = '''SELECT * FROM reviews'''
pd.io.sql.read_sql(query, con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
query  = '''SELECT * FROM publishers'''
pd.io.sql.read_sql(query, con = engine).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Сколько книг вышло после 1 января 2000 года

In [7]:
query_3 = '''SELECT COUNT(title)
            FROM books
            WHERE publication_date >= '2000-01-01'
            '''

result = pd.io.sql.read_sql(query_3, con = engine).iat[0,0]
print(f'После 1 января 2000 года вышло {result} книг')

После 1 января 2000 года вышло 821 книг


## Количество обзоров и средняя оценка для каждой книги 

In [12]:
query_4 = '''SELECT b.*, COUNT(DISTINCT rev) as review_count, AVG(rat.rating) as avg_rating
             FROM  books as b
             LEFT JOIN reviews as rev ON b.book_id = rev.book_id
             LEFT JOIN ratings  as rat ON b.book_id = rat.book_id
             GROUP BY b.book_id'''
result = pd.io.sql.read_sql(query_4, con = engine)
result.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id,review_count,avg_rating
0,1,546,'Salem's Lot,594,2005-11-01,93,2,3.666667
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336,1,2.500000
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135,3,4.666667
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309,2,4.500000
4,5,125,1776,386,2006-07-04,268,4,4.000000


## Издательство, которое выпустило наибольшее число книг толще 50 страниц 
Чтобы исключить из анализа брошюры

In [9]:
query_5 = '''SELECT p.*, COUNT(book_id) as book_count
           FROM publishers as p
           LEFT JOIN books as b ON p.publisher_id = b.publisher_id
           WHERE num_pages > 50      
           GROUP BY p.publisher_id
           ORDER BY COUNT(book_id) DESC
           LIMIT 1 '''

result = pd.io.sql.read_sql(query_5, con = engine).iat[0,1]
print(f'Издательство {result} выпустило наибольшее число книг толще 50 страниц')


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц


## Автор с самой высокой средней оценкой книг 
Учитываются только книги с 50 и более оценками

In [10]:
query_6 = '''WITH one as
            (SELECT author, b.book_id, AVG(rating) AS avg_rating
            FROM authors as a
            LEFT JOIN books as b ON b.author_id = a.author_id
            LEFT JOIN ratings as r ON r.book_id = b.book_id
            GROUP BY author, b.book_id
            HAVING COUNT(rating) > 50)
            SELECT author, AVG(one.avg_rating) AS avg_rtg
            FROM one
            GROUP BY author
            ORDER BY avg_rtg DESC
            LIMIT 1'''

result = pd.io.sql.read_sql(query_6, con = engine).iat[0,0]
print(f'У автора {result} самая высокая средняя оценка книг')

У автора J.K. Rowling/Mary GrandPré самая высокая средняя оценка книг


## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query_7 = '''WITH one as
            (SELECT r.username, COUNT(DISTINCT review_id) as reviews_count
            FROM reviews 
            LEFT JOIN ratings as r ON reviews.username = r.username         
            GROUP BY r.username
            HAVING COUNT(DISTINCT r.book_id) > 50)
            SELECT AVG(reviews_count)
            FROM one'''

result = round(pd.io.sql.read_sql(query_7, con = engine).iat[0,0],1)
print(f'Среднее количество обзоров пользователей, которые поставили больше 50 оценок - {result}')

Среднее количество обзоров пользователей, которые поставили больше 50 оценок - 24.3


## Заключение

Всего, начиная с 2000 года, выпущена 821 книга. <br>
Издательство Penguin Booksу выпустило больше всего книг (исключая выпущенные всеми издательствами брошюры)<br>
Самая высокая оценка пользователей у автора J.K. Rowling/Mary GrandPré. <br>
У самых активных пользователей, которые поставили более 50 оценок, в среднем 24,3 обзора.<br>
На основании этих данных можно готовить предложение для покупателей, опираясь на популярные издательства и любимых авторов пользователей, а также на возможность ставить оценки и писать обзоры.<br>